<a href="https://colab.research.google.com/github/carlibeisel/CDL_analyze_croptype_by_pixel/blob/main/04_landcover_calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: March 14, 2022

Date modified: May 13, 2024

Purpose: Calculates annual percent for each land use from LCMAP data.

##Install Libraries##

You must install the correct version of pylandstats (2.1.3)

In [1]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats==2.1.3 #need this correct version of pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time

import glob
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transonic-0.6.4-py3-none-any.whl (74 kB)
  Using cached autopep8-2.1.1-py2.py3-none-any.whl (45 kB)
  Using cached beniget-0.4.1-py3-none-any.whl (9.4 kB)
  Using cached pycodestyle-2.11.1-py2.py3-none-any.whl (31 kB)
  Created wheel for pylandstats: filename=pylandstats-2.1.3-cp310-cp310-linux_x86_64.whl size=64595 sha256=c62c2391bf2c6eb0e40588301b40c021e6c5fbd4553b769075317d6c1c379d40
  Stored in directory: /root/.cache/pip/wheels/34/f6/d2/09bb553889855a679e8f776a622b9dc3818bdd0182a4e40499
Successfully built pylandstats


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#Scripts for calculating land cover#

In [10]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ## from diversion code
## ---------------------- ##
import rasterio
with rasterio.Env(GTIFF_SRS_SOURCE='EPSG'): #added to project all in same EPSG coordinate system
    shp_file = gpd.read_file('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_Merge_07072022.shp') #open shapefile
    names = shp_file['Name']
    files = glob.glob('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/*.tiff') #get all the years of cdl imagery
    data =[]
    for i in range(len(files)):
        data.append(rasterio.open(files[i])) #open cdl image and append to a list
        print(rasterio.open(files[i])) #added to check
    shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
    years = np.arange(1987, 2021) #years of LCMAP data
    collection = []
    for i in range(len(shp)):
        for n in range(len(years)):
            dataset = data[n]
            year_out = dataset.name[83:87] #change these numbers if printing name wrong due to projection error
            extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
            coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
            out_img, out_transform = mask(dataset=dataset, shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
            out_meta = dataset.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": out_img.shape[1],
                             "width": out_img.shape[2],
                             "transform": out_transform})
            # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
            in_file = files[n]
            out_filename = os.path.join('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
            # Save clipped land cover coverage THIS WILL OVERWRITE FILES
            out_tiff = rasterio.open(out_filename, 'w', **out_meta)
            out_tiff.write(np.squeeze(out_img, 0), 1)
            out_tiff.close()
            collection.append(out_img)

<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1990_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1991_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1992_V12_LCPRI.tiff' mode='r'>
<open DatasetReader 

In [12]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in names:
  files= sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/'+i+'*.tif')) #name for all the csv files
  print(files)
  sta = SpatioTemporalAnalysis(files, dates=years) #import all CDL rasters and mask
  temporal_group.append(sta)


['/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1986.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1987.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1988.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1989.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1990.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1991.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/West Hartley_1992.tif', '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calcu

In [14]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/proportions/'+names[i]+'_prop.csv')
  proportions.append(df)


In [15]:
# ------------------------------- #
#  CALCULATE LARGEST PATCH METRICS
# ------------------------------- #
patch = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['largest_patch_index'])
  df.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/largest_patch_index/'+names[i]+'_patch.csv')
  patch.append(df)

In [16]:
# ------------------------------- #
#     CALCULATE CONTAGION
# ------------------------------- #
# Original code computed contagion + largest_patch_index in the same step. BUt you
# cannot compute contagion with landscape_metrics.
# Make sure you have the correct version of PyLandStats (2.1.3) or it won't run,
# the error code won't look like its a package version issue.
for i in range(len(names)):
    contag = []
    files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/masked_lcmap/'+names[i]+'_*.tif'))
    k = 0
    for file in files:
        sta = pylandstats.Landscape(file, nodata=0)
        DD = sta.contagion()
        output_file = '/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/contagion/'+names[i]+'_contagion.csv'
        contag.append({'dates': years[k], 'contagion': DD})
        k = k+1
        contag_df = pd.DataFrame(contag)
        contag_df.to_csv(output_file, index=False)

In [18]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes
files_proportions = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/proportions/*_prop.csv'))
files_contagion = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/contagion/*_contagion.csv'))
files_patch = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/largest_patch_index/*_patch.csv'))
names = list(sorted(shp_file['Name']))

proportions = []
for i in files_proportions:
  data = pd.read_csv(i)
  proportions.append(data)
contagion = []
for i in files_contagion:
  data = pd.read_csv(i)
  contagion.append(data)
patch = []
for i in files_patch:
  data = pd.read_csv(i)
  patch.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [20]:
## ------------------------------------------ ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING ##
## ------------------------------------------ ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/proportions/change.csv')


In [21]:
## -------------------- ##
## MERGE DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  contag = contagion[i]
  pat = patch[i]
  df_merge = df.merge(contag, on='dates', how='left').merge(pat, on='dates', how='left')
  display (df_merge)
  df_merge.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/final_metrics/'+ names[i] +'.csv')
  merged.append(df_merge)

,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Conway Gulch,2.537427,93.205334,79.527044,92.985424
1,1988,Conway Gulch,2.447207,92.793707,83.489999,92.531506
2,1989,Conway Gulch,2.503595,92.672475,83.317705,92.390538
3,1990,Conway Gulch,2.503595,92.548423,83.225578,92.325693
4,1991,Conway Gulch,2.593814,92.497674,83.223716,92.258028
5,1992,Conway Gulch,2.664298,92.339790,82.991613,92.119879
6,1993,Conway Gulch,2.740421,92.258028,78.448814,92.032479
7,1994,Conway Gulch,2.777073,92.162170,78.464737,92.004285
8,1995,Conway Gulch,2.884209,92.015563,78.171900,91.866137
9,1996,Conway Gulch,2.980067,91.883053,82.624090,91.795653


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Dixie drain,12.237357,87.428122,81.276757,86.559387
1,1988,Dixie drain,12.472761,87.163566,81.135515,86.321068
2,1989,Dixie drain,12.601031,87.024364,80.993552,86.151256
3,1990,Dixie drain,12.723470,86.893179,80.906913,85.988733
4,1991,Dixie drain,12.837163,86.785316,80.861321,85.903463
5,1992,Dixie drain,12.981467,86.654860,80.797236,85.759159
6,1993,Dixie drain,13.181159,86.442049,80.647808,85.592263
7,1994,Dixie drain,13.348784,86.264221,80.533087,85.420265
8,1995,Dixie drain,13.584917,86.019342,82.370396,85.213285
9,1996,Dixie drain,13.850930,85.691380,82.110615,84.947271


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Drainage District No. 3,86.542978,11.995068,76.052771,86.049789
1,1988,Drainage District No. 3,88.586191,9.881400,80.238622,88.233913
2,1989,Drainage District No. 3,90.259512,8.161109,79.768922,90.147957
3,1990,Drainage District No. 3,92.056130,6.405589,82.092452,91.938704
4,1991,Drainage District No. 3,94.052372,4.567872,85.452962,93.940817
5,1992,Drainage District No. 3,95.578910,3.123532,88.249075,95.514326
6,1993,Drainage District No. 3,96.876468,1.861202,90.842154,96.817755
7,1994,Drainage District No. 3,97.704321,1.127290,93.032714,97.651480
8,1995,Drainage District No. 3,98.232738,0.598873,94.481142,98.179897
9,1996,Drainage District No. 3,98.344293,0.469704,94.847994,98.291451


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Eagle Drain,26.877778,17.603777,63.126491,55.078978
1,1988,Eagle Drain,28.088195,16.491299,62.867306,54.925793
2,1989,Eagle Drain,28.781296,15.775596,62.916497,54.913237
3,1990,Eagle Drain,29.836016,14.864016,62.778272,54.654579
4,1991,Eagle Drain,31.104191,13.786695,63.342820,54.541574
5,1992,Eagle Drain,32.289495,12.804801,63.797110,54.375832
6,1993,Eagle Drain,34.444138,10.803345,64.800480,54.215113
7,1994,Eagle Drain,36.475729,8.904849,65.535552,53.471786
8,1995,Eagle Drain,38.771000,6.780342,66.910105,53.338691
9,1996,Eagle Drain,40.315411,5.800959,67.894984,52.884157


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,East Hartley Drain,4.893434,76.934418,67.898630,76.307342
1,1988,East Hartley Drain,5.025942,76.443582,67.696315,75.838901
2,1989,East Hartley Drain,5.104326,76.327871,67.523351,75.725057
3,1990,East Hartley Drain,5.184577,76.098317,67.508196,75.529096
4,1991,East Hartley Drain,5.167780,76.057258,67.609515,75.491770
5,1992,East Hartley Drain,5.305886,75.932216,67.497933,75.390990
6,1993,East Hartley Drain,5.397335,75.760517,67.596378,75.312605
7,1994,East Hartley Drain,5.382405,75.678399,67.899971,75.310739
8,1995,East Hartley Drain,5.443992,75.572020,67.856160,75.226755
9,1996,East Hartley Drain,5.438393,75.510433,67.921951,75.163301


...

,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Sand Run Gulch,2.450304,55.662735,65.945689,46.621721
1,1988,Sand Run Gulch,2.439504,53.407592,67.795248,46.220601
2,1989,Sand Run Gulch,2.437961,53.205489,67.940166,46.137291
3,1990,Sand Run Gulch,2.456475,52.584139,71.694287,45.648619
4,1991,Sand Run Gulch,2.480773,52.331896,71.919765,45.428774
5,1992,Sand Run Gulch,2.543255,52.256686,74.116093,45.401004
6,1993,Sand Run Gulch,2.592238,52.174919,71.854856,45.349322
7,1994,Sand Run Gulch,2.628879,52.159877,71.728268,29.578361
8,1995,Sand Run Gulch,2.636593,52.021414,71.863965,29.517807
9,1996,Sand Run Gulch,2.654720,52.166820,68.484788,30.838804


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,South Middleton,8.095772,57.118953,64.405748,56.035253
1,1988,South Middleton,8.313914,56.746623,64.374945,55.786449
2,1989,South Middleton,8.490004,56.552135,64.213117,55.605978
3,1990,South Middleton,8.631051,56.393566,64.077265,55.468435
4,1991,South Middleton,8.869343,56.154399,63.866040,55.042665
5,1992,South Middleton,9.245177,55.832881,63.549413,54.749181
6,1993,South Middleton,9.623640,55.494717,63.154578,54.375975
7,1994,South Middleton,9.949538,55.194225,62.817690,54.102640
8,1995,South Middleton,10.580309,54.517898,62.316703,52.728085
9,1996,South Middleton,11.003452,54.108773,61.995881,52.326845


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,ThurmanDrain,77.379010,21.959398,73.864968,74.687330
1,1988,ThurmanDrain,79.363785,19.893058,66.853575,76.925866
2,1989,ThurmanDrain,80.650716,18.560812,67.319953,78.294363
3,1990,ThurmanDrain,82.182345,17.038245,68.380500,79.572231
4,1991,ThurmanDrain,84.810585,14.681892,71.380527,83.251767
5,1992,ThurmanDrain,87.466014,12.117093,74.100445,86.931303
6,1993,ThurmanDrain,90.085191,9.543230,80.653405,89.613921
7,1994,ThurmanDrain,91.480877,8.093167,82.511871,91.290556
8,1995,ThurmanDrain,93.520029,6.017763,85.401457,93.501903
9,1996,ThurmanDrain,94.598514,4.930216,87.299256,94.580388


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,West Hartley,3.263314,32.233211,69.190750,63.753450
1,1988,West Hartley,3.270980,30.669273,71.316341,65.265001
2,1989,West Hartley,3.254370,30.578555,71.486811,65.340386
3,1990,West Hartley,3.263314,30.385618,71.699907,65.480936
4,1991,West Hartley,3.268425,30.311510,71.806127,65.540989
5,1992,West Hartley,3.334867,30.199070,71.779268,65.580599
6,1993,West Hartley,3.373198,30.064909,71.983560,65.694317
7,1994,West Hartley,3.430696,30.043187,71.650323,65.640652
8,1995,West Hartley,3.480527,29.787642,71.611526,65.838700
9,1996,West Hartley,3.517582,29.662425,71.566523,65.951140


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Willow Creek,1.495315,13.286905,82.763099,83.807314
1,1988,Willow Creek,1.521139,12.866348,83.465328,84.209836
2,1989,Willow Creek,1.552701,12.800764,83.533146,84.274600
3,1990,Willow Creek,1.584673,12.684352,83.610489,84.358220
4,1991,Willow Creek,1.643289,12.621228,83.619030,84.380764
5,1992,Willow Creek,1.761750,12.553594,83.479542,84.347562
6,1993,Willow Creek,1.828154,12.472024,83.530035,84.377075
7,1994,Willow Creek,1.855617,12.435543,83.571356,84.436921
8,1995,Willow Creek,1.942516,12.445791,83.458337,84.409457
9,1996,Willow Creek,1.985965,12.418737,83.473603,84.393881
